In [9]:
import scipy
import numpy as np
import csv
import cv2
import os
from tqdm import tqdm_notebook as tqdm
import pickle


from scoring.score import score, convert_to_rectangle_list
from scoring.matching import Matching
from scoring.rectangle import Rectangle
from utils.utils import plot_one_box

In [2]:
# Get all ground truth bboxes
mat = scipy.io.loadmat('scoring/ground_truth.mat')
gt_coords, gt_chips, gt_classes = mat['gt_coords'], mat['gt_chips'], mat['gt_classes']
gt_unique = np.unique(gt_classes.astype(np.int64))

In [3]:
# Get class number to name mappings
class_nums = [ 11.0, 12.0, 13.0, 15.0, 17.0, 18.0, 19.0, 20.0, 21.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 40.0, 41.0, 42.0, 44.0, 45.0, 47.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 71.0, 72.0, 73.0, 74.0, 76.0, 77.0, 79.0, 83.0, 84.0, 86.0, 89.0, 91.0, 93.0, 94.0]
with open('data/xview.names') as f:
    lines = f.readlines()

class_num_to_name = { int(c): lines[i].strip() for i, c in enumerate(class_nums) }
print(class_num_to_name)

{11: 'Fixed-wing Aircraft', 12: 'Small Aircraft', 13: 'Cargo Plane', 15: 'Helicopter', 17: 'Passenger Vehicle', 18: 'Small Car', 19: 'Bus', 20: 'Pickup Truck', 21: 'Utility Truck', 23: 'Truck', 24: 'Cargo Truck', 25: 'Truck w/Box', 26: 'Truck Tractor', 27: 'Trailer', 28: 'Truck w/Flatbed', 29: 'Truck w/Liquid', 32: 'Crane Truck', 33: 'Railway Vehicle', 34: 'Passenger Car', 35: 'Cargo Car', 36: 'Flat Car', 37: 'Tank car', 38: 'Locomotive', 40: 'Maritime Vessel', 41: 'Motorboat', 42: 'Sailboat', 44: 'Tugboat', 45: 'Barge', 47: 'Fishing Vessel', 49: 'Ferry', 50: 'Yacht', 51: 'Container Ship', 52: 'Oil Tanker', 53: 'Engineering Vehicle', 54: 'Tower crane', 55: 'Container Crane', 56: 'Reach Stacker', 57: 'Straddle Carrier', 59: 'Mobile Crane', 60: 'Dump Truck', 61: 'Haul Truck', 62: 'Scraper/Tractor', 63: 'Front loader/Bulldozer', 64: 'Excavator', 65: 'Cement Mixer', 66: 'Ground Grader', 71: 'Hut/Tent', 72: 'Shed', 73: 'Building', 74: 'Aircraft Hangar', 76: 'Damaged Building', 77: 'Facility

### Get stats from an image
Adapted from scoring/score.py, `score` function.

In [4]:
output_folder = 'output_30_original'

In [5]:
def get_detections(img_filename):
    with open('./' + output_folder + '/' + img_filename + '.txt', 'r') as f:
        arr = np.array(list(csv.reader(f, delimiter=" ")))
        assert arr.shape[0] != 0 # make sure file isn't empty
        
        arr = arr[:, :6].astype(np.float64)
        threshold = 0
        arr = arr[arr[:, 5] > threshold] # Claire comment: confidence threshold (currently 0)
        det_classes = list(arr[:, 4])
        num_preds = arr.shape[0]

        if np.any(arr[:, :4] < 0):
            raise ValueError('Bounding boxes cannot be negative.')

        if np.any(arr[:, 5] < 0) or np.any(arr[:, 5] > 1):
            raise ValueError('Confidence scores should be between 0 and 1.')
            
        det_box = arr[:, :4]
        det_scores = arr[:, 5]
        det_cls = arr[:, 4]
    
    return det_box, det_scores, det_cls

In [14]:
def evaluate_one(img_filename, iou_threshold=.5):
    assert (iou_threshold < 1 and iou_threshold > 0)
    
    # Get predicted bboxes
    det_box, det_scores, det_cls = get_detections(img_filename)
        
    # Get ground truth bboxes
    gt_box = gt_coords[(gt_chips == img_filename).flatten()]
    gt_cls = gt_classes[(gt_chips == img_filename)]
    
    print('%d detected boxes, %d ground truth boxes' % (len(det_box), len(gt_box)))
    
    per_class_data = {}
    for i in gt_unique:
        per_class_data[i] = {
            'scores': [],
            'rects': [],
            'gt_rects': [],
            'rects_matched': [],
            'gt_rects_matched': [],
        }
    
    # For each class, match predicted and GT bboxes
    for i in gt_unique:
        s = det_scores[det_cls == i] # scores for this class
        ssort = np.argsort(s)[::-1] # sorted score indices
        sorted_scores = s[ssort].tolist() # sorted scores

        # bounding boxes for this class (gt and predicted)
        gt_box_i_cls = gt_box[gt_cls == i].flatten().tolist()
        det_box_i_cls = det_box[det_cls == i]
        det_box_i_cls = det_box_i_cls[ssort].flatten().tolist()

        # convert into Rectangle objects (has helper functions)
        gt_rects, _, _ = convert_to_rectangle_list(gt_box_i_cls)
        rects, _, small_indices = convert_to_rectangle_list(det_box_i_cls)

        # match all Rectangles
        matching = Matching(gt_rects, rects)
        rects_matched, gt_matched = matching.greedy_match(iou_threshold) # Claire comment: returns two boolean lists
        
        
        tp_indices = [i for i in range(len(rects)) if rects_matched[i] == True] if len(rects_matched) > 0 else []
        fp_indices = [i for i in range(len(rects)) if rects_matched[i] == False] if len(rects_matched) > 0 else []
        fn_indices = [i for i in range(len(gt_rects)) if gt_matched[i] == False] if len(gt_matched) > 0 else []
        
        # Rectangles by match type
        tp_rects = [rects[i] for i in tp_indices]
        fp_rects = [rects[i] for i in fp_indices]
        fn_rects = [gt_rects[i] for i in fn_indices]
        
        # Scores by match type
        tp_scores = [sorted_scores[i] for i in tp_indices]
        fp_scores = [sorted_scores[i] for i in fp_indices]
                
            
        per_class_data[i] = {
            'scores': sorted_scores,
            'rects': rects,
            'gt_rects': gt_rects,
            'rects_matched': rects_matched,
            'gt_rects_matched': gt_matched,
            # Counts by match type
            'num_tp': len(tp_rects),
            'num_fp': len(fp_rects),
            'num_fn': len(fn_rects),
            # Rects by match type
            'tp_rects': tp_rects,
            'fp_rects': fp_rects,
            'fn_rects': fn_rects,
            # Scores by match type
            'tp_scores': tp_scores,
            'fp_scores': fp_scores,
        }

    return per_class_data


In [30]:
per_class_data = evaluate_one('1184.tif')

1933 detected boxes, 488 ground truth boxes


In [10]:
print(per_class_data[11])

{'scores': [0.396342], 'rects': [<scoring.rectangle.Rectangle object at 0x7f43620ee908>], 'gt_rects': [], 'rects_matched': [False], 'gt_rects_matched': [], 'true_positives': [], 'false_positives': [<scoring.rectangle.Rectangle object at 0x7f43620ee908>], 'false_negatives': []}


### Draw true positives, false positives, false negatives on img

In [15]:
def plot_bboxes(img_filename, per_class_data):
    img_path = '../../xview_data/test_images/' + img_filename
    img = cv2.imread(img_path)
    
    output_path = 'bboxes_30_original/' + img_filename
    
    # Draw bounding boxes and labels of detections
    for c in per_class_data:
        class_name = class_num_to_name[c] if c in class_num_to_name else 'Unknown'
        data = per_class_data[c]
#         print('%s' % (class_name))
        
        # True positives
        color = [0, 255, 0]
        tp_scores = data['tp_scores']
#         print('\tTrue positives: %d' % (data['num_tp']))
        for i, rect in enumerate(data['tp_rects']):
            xmin, ymin, xmax, ymax = rect.coords
            x1, y1, x2, y2 = max(xmin, 0), max(ymin, 0), max(xmax, 0), max(ymax, 0)
            label = '%s %.2f' % (class_name, tp_scores[i])
            plot_one_box([x1, y1, x2, y2], img, label=label, color=color, line_thickness=1)
        
        # False positives
        color = [0, 0, 255]
        fp_scores = data['fp_scores']
#         print('\tFalse positives: %d' % (data['num_fp']))
        for i, rect in enumerate(data['fp_rects']):
            xmin, ymin, xmax, ymax = rect.coords
            x1, y1, x2, y2 = max(xmin, 0), max(ymin, 0), max(xmax, 0), max(ymax, 0)
            label = '%s %.2f' % (class_name, fp_scores[i])
            plot_one_box([x1, y1, x2, y2], img, label=label, color=color, line_thickness=1)
    
        # False negatives
        color = [255, 0, 0]
#         print('\tFalse negatives: %d' % (data['num_fn']))
        for rect in data['fn_rects']:
            xmin, ymin, xmax, ymax = rect.coords
            x1, y1, x2, y2 = max(xmin, 0), max(ymin, 0), max(xmax, 0), max(ymax, 0)
            plot_one_box([x1, y1, x2, y2], img, label=class_name, color=color, line_thickness=1)
        
#         print()
    
    cv2.imwrite(output_path.replace('.tif', '.jpg'), img)

In [ ]:
plot_bboxes('1184.tif', per_class_data)

### Run for all images

In [16]:
test_imgs = os.listdir('../../xview_data/test_images')

all_data = {}
for file in tqdm(test_imgs):
    if not file.endswith('.tif'):
        continue
        
    per_class_data = evaluate_one(file)
    plot_bboxes(file, per_class_data)
    all_data[file] = per_class_data
    
pickle.dump(all_data, open('test_evaluation.pkl', 'wb'))

152 detected boxes, 36 ground truth boxes






























































1338 detected boxes, 522 ground truth boxes






























































125 detected boxes, 7 ground truth boxes






























































44 detected boxes, 19 ground truth boxes






























































138 detected boxes, 138 ground truth boxes






























































68 detected boxes, 43 ground truth boxes






























































3953 detected boxes, 795 ground truth boxes






























































11 detected boxes, 1 ground truth boxes






























































32 detected boxes, 10 ground truth boxes






























































458 detected boxes, 14 ground truth boxes

























14281 detected boxes, 3674 ground truth boxes






























































4477 detected boxes, 2457 ground truth boxes






























































2012 detected boxes, 676 ground truth boxes






























































13 detected boxes, 2 ground truth boxes






























































3223 detected boxes, 555 ground truth boxes






























































1489 detected boxes, 462 ground truth boxes

































































In [17]:
test_data = pickle.load(open('test_evaluation.pkl', 'rb'))

In [22]:
# Write to csv
with open('test_evaluation.csv', mode='w') as file:
    file_writer = csv.writer(file, delimiter=',')
    
    for img in tqdm(test_data):
        for c in test_data[img]:
            class_name = class_num_to_name[c] if c in class_num_to_name else 'Unknown'
            data = test_data[img][c]
            
            tp_scores = data['tp_scores']
            for i, rect in enumerate(data['tp_rects']):
                file_writer.writerow([
                    img,
                    class_name,
                    rect.area(),
                    'tp',
                    tp_scores[i]
                ])
                
            fp_scores = data['fp_scores']
            for i, rect in enumerate(data['fp_rects']):
                file_writer.writerow([
                    img,
                    class_name,
                    rect.area(),
                    'fp',
                    fp_scores[i]
                ])
                
            for i, rect in enumerate(data['fn_rects']):
                file_writer.writerow([
                    img,
                    class_name,
                    rect.area(),
                    'fn',
                    0
                ])
        

In [33]:
total_num_gt = 0

for c in test_data['1799.tif']:
    data = test_data['1799.tif'][c]
#     print('%d: %d == %d + %d' % (c, len(data['gt_rects']), data['num_tp'], data['num_fn']))
    assert(len(data['gt_rects']) == data['num_tp'] + data['num_fn'])
    total_num_gt += len(data['gt_rects'])
    
print(total_num_gt)

43
